In [ ]:
!pip -q install langchain huggingface_hub transformers sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.1/489.1 KB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 64.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 6.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 78.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 KB 6.3 MB/s eta 0:00:00


## HuggingFace

There are two Hugging Face LLM wrappers, one for a local pipeline and one for a model hosted on Hugging Face Hub. Note that these wrappers only work for models that support the following tasks: text2text-generation, text-generation


In [ ]:
import os


os.environ['HUGGINGFACEHUB_API_TOKEN'] = ''

## Use the HuggingFaceHub

In [ ]:
from langchain import PromptTemplate, HuggingFaceHub, LLMChain

template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate(template=template, input_variables=["question"])

In [ ]:
llm_chain = LLMChain(prompt=prompt,
                     llm=HuggingFaceHub(repo_id="google/flan-t5-xl",
                                        model_kwargs={"temperature":0,
                                                      "max_length":64}))

In [ ]:
question = "What is the capital of France?"

print(llm_chain.run(question))

Paris is the capital of France. The final answer: Paris.


In [ ]:
question = "What area is best for growing wine in France?"

print(llm_chain.run(question))

The best area for growing wine in France is the Loire Valley. The Loire Valley is located in the south of France. The area of France that is best for growing wine is the Loire Valley. The final answer: Loire Valley.


## BlenderBot

Doesn't work on the Hub

In [ ]:
blenderbot_chain = LLMChain(prompt=prompt,
                     llm=HuggingFaceHub(repo_id="facebook/blenderbot-1B-distill",
                                        model_kwargs={"temperature":0,
                                                      "max_length":64}))

ValidationError: ignored

In [ ]:
# question = "What is the capital of France?"
# question = "What area is best for growing wine in France?"

# print(blenderbot_chain = LLMChain(prompt=prompt,
# .run(question))

## With Local model from HF

### Why would you want to use local mode?

- fine-tuned models
- GPU hosted etc
- some models only work locally

## T5-Flan - Encoder-Decoder

In [ ]:
from langchain.llms import HuggingFacePipeline
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, AutoModelForSeq2SeqLM

model_id = 'google/flan-t5-large'# go for a smaller model if you dont have the VRAM
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id, load_in_8bit=False)

pipe = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=100
)

local_llm = HuggingFacePipeline(pipeline=pipe)


In [ ]:
print(local_llm('What is the capital of France? '))

paris


In [ ]:
llm_chain = LLMChain(prompt=prompt,
                     llm=local_llm
                     )

question = "What is the capital of England?"

print(llm_chain.run(question))

The capital of England is London. London is the capital of England. So the answer is London.


## GPT2-medium - Decoder Only Model

microsoft/DialoGPT-large

In [ ]:
model_id = "gpt2-medium"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=100
)

local_llm = HuggingFacePipeline(pipeline=pipe)

In [ ]:
llm_chain = LLMChain(prompt=prompt,
                     llm=local_llm
                     )

question = "What is the capital of France?"

print(llm_chain.run(question))

/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1201: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 The first word, Paris, was given by the Bishop of Paris in 796. In 793, the bishop gave an edict that gave its capital for the moment to Louis I of France and to Philip V of France; this decree was a little complicated. France's name came from the French nation. Louis was the first king of France and was known as France's first king.




## BlenderBot - Encoder-Decoder

In [ ]:
from langchain.llms import HuggingFacePipeline
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, AutoModelForSeq2SeqLM

model_id = 'facebook/blenderbot-1B-distill'
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

pipe = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=100
)

local_llm = HuggingFacePipeline(pipeline=pipe)

In [ ]:
llm_chain = LLMChain(prompt=prompt,
                     llm=local_llm
                     )

question = "What area is best for growing wine in France?"

print(llm_chain.run(question))

 I'm not sure, but I do know that France is one of the largest producers of wine in the world.


## SentenceTransformers

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

model_name = "sentence-transformers/all-mpnet-base-v2"

hf = HuggingFaceEmbeddings(model_name=model_name)

In [ ]:
hf.embed_query('this is an embedding')

[0.010657327249646187,
 -0.09967267513275146,
 -0.026967084035277367,
 0.0653177872300148,
 0.02100495621562004,
 0.042623456567525864,
 0.011534163728356361,
 -0.006229322869330645,
 0.05175820365548134,
 0.007306752726435661,
 0.02135349065065384,
 0.04269150272011757,
 0.023143865168094635,
 0.009952719323337078,
 0.056463081389665604,
 -0.06137979030609131,
 0.05274379253387451,
 0.024683983996510506,
 -0.013267713598906994,
 -0.007051217835396528,
 0.026656342670321465,
 -0.0059135290794074535,
 0.004097490105777979,
 0.03841235488653183,
 -0.014230660162866116,
 0.02302352711558342,
 -0.007326637394726276,
 -0.035625338554382324,
 -0.017934121191501617,
 -0.01393020898103714,
 0.011977488175034523,
 -0.0073659527115523815,
 0.024451538920402527,
 -0.06637248396873474,
 1.5677644569223048e-06,
 0.01821720413863659,
 0.0019748671911656857,
 -0.01832936331629753,
 -0.014930741861462593,
 -0.005393384024500847,
 -0.01122234109789133,
 0.015792902559041977,
 -0.02714185230433941,
 -0.

In [ ]:
hf.embed_documents(['this is an embedding','this another embedding'])

[[0.010657311417162418,
  -0.09967269003391266,
  -0.026967095211148262,
  0.06531775742769241,
  0.02100495994091034,
  0.04262346029281616,
  0.011534159071743488,
  -0.0062293317168951035,
  0.05175821855664253,
  0.007306770421564579,
  0.021353470161557198,
  0.04269150644540787,
  0.023143867030739784,
  0.009952726773917675,
  0.056463103741407394,
  -0.06137977913022041,
  0.0527438223361969,
  0.02468400076031685,
  -0.013267757371068,
  -0.007051236927509308,
  0.026656322181224823,
  -0.005913532804697752,
  0.004097457975149155,
  0.03841238096356392,
  -0.014230652712285519,
  0.023023530840873718,
  -0.007326613180339336,
  -0.03562537208199501,
  -0.01793413795530796,
  -0.013930226676166058,
  0.01197753194719553,
  -0.00736597366631031,
  0.024451516568660736,
  -0.06637248396873474,
  1.5677645706091425e-06,
  0.018217213451862335,
  0.001974882557988167,
  -0.018329348415136337,
  -0.014930753037333488,
  -0.005393414292484522,
  -0.011222323402762413,
  0.0157929323

In [ ]:


hf = HuggingFaceHubEmbeddings(
    repo_id=model_name,
    task="feature-extraction",
    # huggingfacehub_api_token="my-api-key",
)